# Abstract

This study aims to identify specific price patterns analyzing high-frequency tick-by-tick data of the carbon emission allowances of companies participating in the European emissions trading system market (European Carbon Emission Allowance), resampling it into 5-minute intervals. We identified innovative patterns in the data and extracted key features from them in order to infer anticipatory knowledge in the data evolution. Based on the average frequency of detected patterns each month, we constructed a dataset consisting of sequences of patterns and their corresponding main features. Using a Generative Adversarial Network (GAN) model combined with Long Short-Term Memory (LSTM) and Self-Attention layers, we were able to generate synthetic sequences. These sequences were then used to produce specific artificial interval data. The quality of the generated dataset was evaluated using key financial time-series metrics, including linear unpredictability, heavy-tailed price return distribution, volatility clustering, leverage effects, coarse-fine volatility correlation, and gain-loss asymmetry. The final results demonstrate that our synthetic data successfully emulates the statistical characteristics of a real financial market. The findings of this study can be applied to various areas of financial analysis, such as risk management, algorithmic trading, market simulation, and stress testing under different market conditions.

# Import needed libraries


In [ ]:
import torch
import random
import numpy as np
import sys
import torch.optim as optim
import json
import os
import pickle
import matplotlib.pyplot as plt
import mplfinance as mpf
import pandas as pd
from my_zigzag import *
import copy

# Inputs


In [4]:
# current_dir = os.path.dirname(os.path.abspath(__file__))
# parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
path = './savingFolder/dataGeneration/data/'
savingPath = './savingFolder/dataGeneration/runs/'
sourcePath = './data/test5_5m.csv'


# Data


In [5]:
sorceData = pd.read_csv(sourcePath)

sorceData.head

<bound method NDFrame.head of         Unnamed: 0                 Date  Volume       Open   High        Low  \
0                0  2018-01-02 08:05:00     159   8.050000   8.05   8.010000   
1                1  2018-01-02 08:10:00     108   8.020000   8.02   8.000000   
2                2  2018-01-02 08:15:00       5   7.990000   7.99   7.990000   
3                3  2018-01-02 08:25:00      11   7.990000   7.99   7.990000   
4                4  2018-01-02 08:35:00      77   7.990000   7.99   7.980000   
...            ...                  ...     ...        ...    ...        ...   
160811      160811  2023-06-09 14:00:00     113  84.870000  85.07  84.832143   
160812      160812  2023-06-09 14:05:00      25  84.980000  85.07  84.980000   
160813      160813  2023-06-09 14:10:00      36  85.050000  85.25  85.050000   
160814      160814  2023-06-09 14:15:00      29  85.251111  85.35  85.251111   
160815      160815  2023-06-09 14:20:00      47  85.270000  85.27  85.126250   

         

# Reproducability Function


In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Define device

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Algorithm1: Pivot and Pattern Detection

In [ ]:
class addZigzagFeatures:

    def __init__(self, data, upThreshold, downThreshold, selectedColumn,\
                 numberOfPivotsInAWave= 5, pivotRatioList = ['W5', 'W4'], zigzagAccordingToWaveLengthSwitch = False):
        
        self. selectedColumn = selectedColumn
        self. data = data
        self. dataCopy = copy.deepcopy(data)
        self. allCloseprice = copy.deepcopy(data)[self. selectedColumn]
        self. upThreshold = upThreshold
        self. downThreshold = downThreshold
        self. numberOfPivotsInAWave = numberOfPivotsInAWave
        self. zigzagAccordingToWaveLengthSwitch = zigzagAccordingToWaveLengthSwitch
        self. allPivotIndexs = list()
        self. dictThatRlMustDecide = dict()
        self. pivotRatioList = pivotRatioList
        self. allStructures = dict()
        self. allLengthSortedStructures = dict()
        self. allStructureWithLengthBasedSortUntillW4 = list()
        self. allStructureWithLengthBasedSortUntillW5  = list()
        self. numberofstructer = 0 
        self. numberofstructer1 = 0
        self. numberOfPivots = 0
        self. dictForTrainInfo = dict()
        self. detectedPossiblePivots = list()
        self. possiblePivots = list()
        self. W5PrimeIndex = list()
        self. totalFeatures = list()
        self. allStructureTrends = list()
        self. allStructuresIndexs = list()
        ############# run functions ################
        self. detectFirstZiroToFivePivots()
        self. mainLoop()
      


    def detectFirstZiroToFivePivots(self):
        self. allPivots = peak_valley_pivots(self. allCloseprice, \
                                             self. upThreshold, self. downThreshold )
    
        self. findIndexOfAllPositivePivots = list(np.arange(len(self. allCloseprice))[self. allPivots == 1])
        self. findIndexOfAllNegativePivots = list(np.arange(len(self. allCloseprice))[self. allPivots == -1])
        self. findIndexOfAllPivots = (self. findIndexOfAllPositivePivots + self. findIndexOfAllNegativePivots)
        self. findIndexOfAllPivots.sort()
        self. findIndexOfFirstFivthPivots = self. findIndexOfAllPivots[self. numberOfPivotsInAWave]
        self. allPivotIndexs = self. findIndexOfAllPivots[0:self. numberOfPivotsInAWave + 1]
    
    def calculateHighAndLowThresholdForAW4(self, W3, W4):

        if self. zigzagAccordingToWaveLengthSwitch == False:
            
            if W4 - W3 > 0 :
                highThreshold =  W4 
                downThreshold = W4 / (1 / (1 - self. upThreshold))
               
            
            else:     
                highThreshold =  W4 / (1/(1 + self. upThreshold))
                downThreshold = W4
        
        if self. zigzagAccordingToWaveLengthSwitch == True:
            
            if W4 - W3 > 0 :
                highThreshold =  W4
                downThreshold = W4 - ((W4 - W3) /  (1 / self. upThreshold))
            
            else:
                highThreshold =  W4 + ((W3 - W4) /  (1/self. upThreshold))
                downThreshold = W4
        
        
        return(highThreshold, downThreshold)
    
    def checkForPassingThreshold(self, wavwType, highThreshold, downThreshold, index):
        

        if downThreshold <= self. dataCopy[self. selectedColumn].values[index] <= highThreshold:
               
                return(False)

        else:
            if wavwType == 'increasingly' and \
                self. dataCopy[self. selectedColumn].values[index] < downThreshold:
                    return('increaseDown')
        
            if wavwType == 'increasingly' and \
                self. dataCopy[self. selectedColumn].values[index] > highThreshold:
                    return('increaseUp')
                    
            if wavwType == 'decreasingly' and \
                self. dataCopy[self. selectedColumn].values[index] < downThreshold:
                    return('decreasDown')
        
            if wavwType == 'decreasingly' and \
                self. dataCopy[self. selectedColumn].values[index] > highThreshold:
                    return('decreasUp')

    
    
    def structreAndLengthSortedCalculation(self, pivots, index, \
                                           structureCollectionSwith = False):

        randomEpsilon = np.random.rand(len(pivots)) * 0.000001
        pivots = list(pivots + randomEpsilon)
        sortedPivots = list(np.sort(pivots))
        structurName = ''

        for item in sortedPivots:
            structurName += str(pivots.index(item))
        
        if structureCollectionSwith == True:
            if structurName not in list(self. allStructures.keys()):
                self. allStructures[structurName] = [index]
                self. numberofstructer += 1
                
            
            else:
                self. allStructures[structurName].append(index)
            
    
        lengthList = list()

        for i in range(1, len(pivots)):
            lengthList.append(abs(pivots[i] - pivots[i-1]))
        
        sortedLength = list(np.sort(lengthList))
    
        structurNameForSort = ''

        for item in sortedLength:
            structurNameForSort += str(lengthList.index(item))
        
        if structurNameForSort not in list(self. allLengthSortedStructures.keys()):
            self. allLengthSortedStructures[structurNameForSort] = [index]
            self. numberofstructer1 += 1
            
        
        else:
            self. allLengthSortedStructures[structurNameForSort].append(index)
        

        return(structurName, structurNameForSort)


    def mainLoop(self):
        W5Prime = self. dataCopy[self. selectedColumn].values[self. findIndexOfFirstFivthPivots+1]
        W4Prime = self. dataCopy[self. selectedColumn].values[self. allPivotIndexs[-1]]
        W3Prime = self. dataCopy[self. selectedColumn].values[self. allPivotIndexs[-2]]
        W2Prime = self. dataCopy[self. selectedColumn].values[self. allPivotIndexs[-3]]
        W1Prime = self. dataCopy[self. selectedColumn].values[self. allPivotIndexs[-4]]
        W0Prime = self. dataCopy[self. selectedColumn].values[self. allPivotIndexs[-5]]
        WDeleted = W0Prime

        
        self. possiblePivots = copy.deepcopy(self. allPivotIndexs)

        highThreshold, downThreshold = self. calculateHighAndLowThresholdForAW4(W3Prime, W4Prime)
        
        flage = False
        calculator = 0
        self. numberOfPivots += 1
        for index in range(self. findIndexOfFirstFivthPivots+1, len(self. dataCopy)):
            print(index)
            if flage == False or index == self. findIndexOfFirstFivthPivots+1:
                
                if W4Prime - W3Prime > 0:
                    wavwType = 'increasingly'
                else :
                    wavwType = 'decreasingly'

           
            if flage == False : flage = self. checkForPassingThreshold(wavwType, highThreshold, downThreshold, index)
            
            if flage == None:
                import sys
                print('flage is not acceptable')
                sys.exit()

            if flage == False:
                W5Prime = self. dataCopy[self. selectedColumn].values[index]
                self. pivotsIndexForPlot[-1] = index

            
            else:
                if flage == 'increaseDown':
                    if self. dataCopy[self. selectedColumn].values[index] > self. dataCopy[self. selectedColumn].values[index-1]:
                        flage = False

                        #### update pivots ##############
                        WDeleted = W0Prime 
                        W0Prime = W1Prime
                        W1Prime = W2Prime
                        W2Prime = W3Prime
                        W3Prime = W4Prime
                        W4Prime = W5Prime
                        W5Prime = self. dataCopy[self. selectedColumn].values[index]
                        self. pivotsIndexForPlot = self. pivotsIndexForPlot[1:]
                        self. pivotsIndexForPlot.append(index)
                        self. allPivotIndexs.append(index-1)
                        self. possiblePivots.append(index - 1)
                        self. W5PrimeIndex.append(index)
                        ##### update thresholds ################
                        highThreshold, downThreshold = self. calculateHighAndLowThresholdForAW4(W3Prime,W4Prime)

                        ######## structure and lengthBasedSor calculation ###############
                        pivotsUntilW4 = [WDeleted, W0Prime, W1Prime, W2Prime, W3Prime, W4Prime]
                        structreUntilW4, sortedLengthStructureUntilW4 = self. structreAndLengthSortedCalculation(pivotsUntilW4, index)
                        totallNameUntillW4 = f'{structreUntilW4}{sortedLengthStructureUntilW4}'

                        ############# saving for new process until W4 ############################
                        structureIndex = self. pivotsIndexForPlot[0:-1]
                        self. allStructureTrends.append(structreUntilW4)
                        self. allStructuresIndexs.append(structureIndex)
                        ###################################################################



                        if totallNameUntillW4 not in self. allStructureWithLengthBasedSortUntillW4:
                            self. allStructureWithLengthBasedSortUntillW4.append(totallNameUntillW4)
                        
                        numberOfAllStructureWithLengthBasedSortUntillW4 = self. allStructureWithLengthBasedSortUntillW4.index(totallNameUntillW4)


                        pivotsUntilW5 = [W0Prime, W1Prime, W2Prime, W3Prime, W4Prime, W5Prime]
                        structreUntilW5, sortedLengthStructureUntilW5 = self. structreAndLengthSortedCalculation(pivotsUntilW5, index, structureCollectionSwith=True)
                        totallNameUntillW5 = f'{structreUntilW5}{sortedLengthStructureUntilW5}'
                        if totallNameUntillW5 not in self. allStructureWithLengthBasedSortUntillW5:
                            self. allStructureWithLengthBasedSortUntillW5.append(totallNameUntillW5)
                        numberOfAllStructureWithLengthBasedSortUntillW5 = self. allStructureWithLengthBasedSortUntillW5.index(totallNameUntillW5)
                        ####### pivot calculator ##############
                        self. numberOfPivots += 1
                        ##### feature calculation #################
                        pointsForFeature = [W0Prime, W1Prime, W2Prime, W3Prime, W4Prime, W5Prime]             
                        ################## should save documents for Rl decision ##############
                        info = dict()
                        info['waveType'] = 'increasingly' if W4Prime > W3Prime else 'decreasingly'
                        info['index'] = int(index)
                        info['pivots'] = [WDeleted, W0Prime, W1Prime, W2Prime, W3Prime, W4Prime, W5Prime] 
                        info['indexOfW5prime'] = index
                        info['structureUntilW4'] = structreUntilW4
                        info['structureUntilW5'] = structreUntilW5
                        info['sortedLengthStructureUntilW4'] = sortedLengthStructureUntilW4
                        info['sortedLengthStructureUntilW5'] = sortedLengthStructureUntilW5
                        info['highThreshold'] = highThreshold
                        info['downThreshold'] = downThreshold
                        info['numberOfAllStructureWithLengthBasedSortUntillW4'] = [int(item) for item in format(numberOfAllStructureWithLengthBasedSortUntillW4, '012b')]
                        info['numberOfAllStructureWithLengthBasedSortUntillW5'] = [int(item) for item in format(numberOfAllStructureWithLengthBasedSortUntillW5, '012b')]
                        self. dictForTrainInfo[f'{len(self. dictForTrainInfo)}'] = info

                        #######################################################
                    else:
                        W5Prime = self. dataCopy[self. selectedColumn].values[index]
                        self. pivotsIndexForPlot[-1] = index
                        pointsForFeature = [W0Prime, W1Prime, W2Prime, W3Prime, W4Prime, W5Prime]             
                

                        
                
                if flage == 'increaseUp':
                    
                    if self. dataCopy[self. selectedColumn].values[index] < self. dataCopy[self. selectedColumn].values[index-1]:
                        flage = False

                        ####### update pivots ##### 
                        WDeleted = W0Prime 
                        W0Prime = W1Prime
                        W1Prime = W2Prime
                        W2Prime = W3Prime
                        W3Prime = W4Prime
                        W4Prime = W5Prime
                        W5Prime = self. dataCopy[self. selectedColumn].values[index]
                        self. pivotsIndexForPlot = self. pivotsIndexForPlot[1:]
                        self. pivotsIndexForPlot.append(index)
                        self. allPivotIndexs.append(index-1)
                        self. possiblePivots.append(index - 1)
                        self. W5PrimeIndex.append(index)
                        ############ threshold calculation ################
                        highThreshold, downThreshold = self. calculateHighAndLowThresholdForAW4(W3Prime,W4Prime)
                        calculator = 0

                        ################ structure calculation #########################
                        pivotsUntilW4 = [WDeleted, W0Prime, W1Prime, W2Prime, W3Prime, W4Prime]
                        structreUntilW4, sortedLengthStructureUntilW4 = self. structreAndLengthSortedCalculation(pivotsUntilW4, index)
                        totallNameUntillW4 = f'{structreUntilW4}{sortedLengthStructureUntilW4}'

                        ############# saving for new process until W4 ############################
                        structureIndex = self. pivotsIndexForPlot[0:-1]
                        self. allStructureTrends.append(structreUntilW4)
                        self. allStructuresIndexs.append(structureIndex)
                        ###################################################################



                        if totallNameUntillW4 not in self. allStructureWithLengthBasedSortUntillW4:
                            self. allStructureWithLengthBasedSortUntillW4.append(totallNameUntillW4)
                        
                        numberOfAllStructureWithLengthBasedSortUntillW4 = self. allStructureWithLengthBasedSortUntillW4.index(totallNameUntillW4)


                        pivotsUntilW5 = [W0Prime, W1Prime, W2Prime, W3Prime, W4Prime, W5Prime]
                        structreUntilW5, sortedLengthStructureUntilW5 = self. structreAndLengthSortedCalculation(pivotsUntilW5, index, structureCollectionSwith=True)
                        
                        ######### patternterend calculation uttil W5 ####################
                        # structureIndex = self. pivotsIndexForPlot[1:]
                        # self. allStructureTrends.append(structreUntilW5)
                        # self. allStructuresIndexs.append(structureIndex)
                        #############################################################
                        
                        totallNameUntillW5 = f'{structreUntilW5}{sortedLengthStructureUntilW5}'
                        if totallNameUntillW5 not in self. allStructureWithLengthBasedSortUntillW5:
                            self. allStructureWithLengthBasedSortUntillW5.append(totallNameUntillW5)
                        numberOfAllStructureWithLengthBasedSortUntillW5 = self. allStructureWithLengthBasedSortUntillW5.index(totallNameUntillW5)
                        ######################## pivot calculator #####################
                        self. numberOfPivots += 1

                        #################### feature calcualation ################

                        ################## should save documents for Rl decision ##############
                        info = dict()
                        info['waveType'] = 'increasingly' if W4Prime > W3Prime else 'decreasingly'
                        info['index'] = int(index)
                        info['pivots'] = [WDeleted,W0Prime, W1Prime, W2Prime, W3Prime, W4Prime, W5Prime] 
                        info['indexOfW5prime'] = index
                        info['structureUntilW4'] = structreUntilW4
                        info['structureUntilW5'] = structreUntilW5
                        info['sortedLengthStructureUntilW4'] = sortedLengthStructureUntilW4
                        info['sortedLengthStructureUntilW5'] = sortedLengthStructureUntilW5
                        info['highThreshold'] = highThreshold
                        info['downThreshold'] = downThreshold
                        info['numberOfAllStructureWithLengthBasedSortUntillW4'] = [int(item) for item in format(numberOfAllStructureWithLengthBasedSortUntillW4, '012b')]
                        info['numberOfAllStructureWithLengthBasedSortUntillW5'] = [int(item) for item in format(numberOfAllStructureWithLengthBasedSortUntillW5, '012b')]
                        self. dictForTrainInfo[f'{len(self. dictForTrainInfo)}'] = info


                        #######################################################
   
                    else:
                        if calculator > 0:
                            W5Prime = self. dataCopy[self. selectedColumn].values[index]
                            self. pivotsIndexForPlot[-1] = index           
                        
                        
                        else:
                            W5Prime = self. dataCopy[self. selectedColumn].values[index]
                            W4Prime = W3Prime
                            W3Prime = W2Prime
                            W2Prime = W1Prime
                            W1Prime = W0Prime
                            W0Prime = WDeleted
                            calculator += 1
                            self. pivotsIndexForPlot[-1] = index
                            self. pivotsIndexForPlot[1:-1] = self. pivotsIndexForPlot[0:-2]
                            del self. allPivotIndexs[-1]

                if flage == 'decreasDown':
                     
                    if self. dataCopy[self. selectedColumn].values[index] > self. dataCopy[self. selectedColumn].values[index-1]:
                        flage = False

                        ######## update pivots ##################
                        WDeleted = W0Prime 
                        W0Prime = W1Prime
                        W1Prime = W2Prime
                        W2Prime = W3Prime
                        W3Prime = W4Prime
                        W4Prime = W5Prime
                        W5Prime = self. dataCopy[self. selectedColumn].values[index]
                        self. pivotsIndexForPlot = self. pivotsIndexForPlot[1:]
                        self. pivotsIndexForPlot.append(index)
                        self. allPivotIndexs.append(index - 1)
                        self. possiblePivots.append(index - 1)
                        self. W5PrimeIndex.append(index)
                        ################ threshold calculation #########################
                        highThreshold, downThreshold = self. calculateHighAndLowThresholdForAW4(W3Prime,W4Prime)
                        calculator = 0
                        ################## structure calculation ############
                        pivotsUntilW4 = [WDeleted, W0Prime, W1Prime, W2Prime, W3Prime, W4Prime]
                        structreUntilW4, sortedLengthStructureUntilW4 = self. structreAndLengthSortedCalculation(pivotsUntilW4, index)
                        totallNameUntillW4 = f'{structreUntilW4}{sortedLengthStructureUntilW4}'

                        ############# saving for new process until W4 ############################
                        structureIndex = self. pivotsIndexForPlot[0:-1]
                        self. allStructureTrends.append(structreUntilW4)
                        self. allStructuresIndexs.append(structureIndex)
                        ###################################################################




                        if totallNameUntillW4 not in self. allStructureWithLengthBasedSortUntillW4:
                            self. allStructureWithLengthBasedSortUntillW4.append(totallNameUntillW4)
                        
                        numberOfAllStructureWithLengthBasedSortUntillW4 = self. allStructureWithLengthBasedSortUntillW4.index(totallNameUntillW4)


                        pivotsUntilW5 = [W0Prime, W1Prime, W2Prime, W3Prime, W4Prime, W5Prime]
                        structreUntilW5, sortedLengthStructureUntilW5 = self. structreAndLengthSortedCalculation(pivotsUntilW5, index, structureCollectionSwith=True)
                        
                        
                        # ######################## pattern calculationuntil W5 ########################
                        # structureIndex = self. pivotsIndexForPlot[1:]
                        # self. allStructureTrends.append(structreUntilW5)
                        # self. allStructuresIndexs.append(structureIndex)
                        ##########################################################################
                        
                        totallNameUntillW5 = f'{structreUntilW5}{sortedLengthStructureUntilW5}'
                        if totallNameUntillW5 not in self. allStructureWithLengthBasedSortUntillW5:
                            self. allStructureWithLengthBasedSortUntillW5.append(totallNameUntillW5)
                        numberOfAllStructureWithLengthBasedSortUntillW5 = self. allStructureWithLengthBasedSortUntillW5.index(totallNameUntillW5)

                        ################## pivot calculator ###########
                        self. numberOfPivots += 1

                        ################## should save documents for Rl decision ##############
                        info = dict()
                        info['waveType'] = 'increasingly' if W4Prime > W3Prime else 'decreasingly'
                        info['index'] = int(index)
                        info['pivots'] = [WDeleted,W0Prime, W1Prime, W2Prime, W3Prime, W4Prime, W5Prime] 
                   
                        info['indexOfW5prime'] = index
                        info['structureUntilW4'] = structreUntilW4
                        info['structureUntilW5'] = structreUntilW5
                        info['sortedLengthStructureUntilW4'] = sortedLengthStructureUntilW4
                        info['sortedLengthStructureUntilW5'] = sortedLengthStructureUntilW5
                        info['highThreshold'] = highThreshold
                        info['downThreshold'] = downThreshold
                        info['numberOfAllStructureWithLengthBasedSortUntillW4'] = [int(item) for item in format(numberOfAllStructureWithLengthBasedSortUntillW4, '012b')]
                        info['numberOfAllStructureWithLengthBasedSortUntillW5'] = [int(item) for item in format(numberOfAllStructureWithLengthBasedSortUntillW5, '012b')]
                        self. dictForTrainInfo[f'{len(self. dictForTrainInfo)}'] = info

                        #################################################################
                    
                    else:
                        if calculator > 0 :
                            W5Prime = self. dataCopy[self. selectedColumn].values[index]
                            self. pivotsIndexForPlot[-1] = index


                        else:

                            W5Prime = self. dataCopy[self. selectedColumn].values[index]
                            W4Prime = W3Prime
                            W3Prime = W2Prime
                            W2Prime = W1Prime
                            W1Prime = W0Prime
                            W0Prime = WDeleted
                            self. pivotsIndexForPlot[-1] = index
                            self. pivotsIndexForPlot[1:-1] = self. pivotsIndexForPlot[0:-2]
                            calculator += 1
                            del self. allPivotIndexs[-1]
  


                if flage == 'decreasUp':
                    if self. dataCopy[self. selectedColumn].values[index] < self. dataCopy[self. selectedColumn].values[index-1]:
                        flage = False

                        ###### update pivots ################
                        WDeleted = W0Prime 
                        W0Prime = W1Prime
                        W1Prime = W2Prime
                        W2Prime = W3Prime
                        W3Prime = W4Prime
                        W4Prime = W5Prime
                        W5Prime = self. dataCopy[self. selectedColumn].values[index]
                        self. allPivotIndexs.append(index - 1)
                        self. possiblePivots.append(index - 1)
                        self. pivotsIndexForPlot = self. pivotsIndexForPlot[1:]
                        self. pivotsIndexForPlot.append(index)
                        self. W5PrimeIndex.append(index)
                        ############ threshold calculation ######################
                        highThreshold, downThreshold = self. calculateHighAndLowThresholdForAW4(W3Prime,W4Prime)

                        ###########  structure calculation ##################
                        pivotsUntilW4 = [WDeleted, W0Prime, W1Prime, W2Prime, W3Prime, W4Prime]
                        structreUntilW4, sortedLengthStructureUntilW4 = self. structreAndLengthSortedCalculation(pivotsUntilW4, index)
                        totallNameUntillW4 = f'{structreUntilW4}{sortedLengthStructureUntilW4}'

                        ############# saving for new process until W4 ############################
                        structureIndex = self. pivotsIndexForPlot[0:-1]
                        self. allStructureTrends.append(structreUntilW4)
                        self. allStructuresIndexs.append(structureIndex)
                        ###################################################################



                        if totallNameUntillW4 not in self. allStructureWithLengthBasedSortUntillW4:
                            self. allStructureWithLengthBasedSortUntillW4.append(totallNameUntillW4)
                        
                        numberOfAllStructureWithLengthBasedSortUntillW4 = self. allStructureWithLengthBasedSortUntillW4.index(totallNameUntillW4)


                        pivotsUntilW5 = [W0Prime, W1Prime, W2Prime, W3Prime, W4Prime, W5Prime]
                        structreUntilW5, sortedLengthStructureUntilW5 = self. structreAndLengthSortedCalculation(pivotsUntilW5, index, structureCollectionSwith=True)
                        
                        
                        ############################ pattern terend calculation until W5 ############
                        # structureIndex = self. pivotsIndexForPlot[1:]
                        # self. allStructureTrends.append(structreUntilW5)
                        # self. allStructuresIndexs.append(structureIndex)
                        ###########################################################################

                        totallNameUntillW5 = f'{structreUntilW5}{sortedLengthStructureUntilW5}'
                        if totallNameUntillW5 not in self. allStructureWithLengthBasedSortUntillW5:
                            self. allStructureWithLengthBasedSortUntillW5.append(totallNameUntillW5)
                        numberOfAllStructureWithLengthBasedSortUntillW5 = self. allStructureWithLengthBasedSortUntillW5.index(totallNameUntillW5)
                        ################### pivot calculator ###############
                        self. numberOfPivots += 1


                        ################## should save documents for Rl decision ##############
                        info = dict()
                        info['waveType'] = 'increasingly' if W4Prime > W3Prime else 'decreasingly'
                        info['index'] = int(index)
                        info['pivots'] = [WDeleted,W0Prime, W1Prime, W2Prime, W3Prime, W4Prime, W5Prime] 
                        info['indexOfW5prime'] = index
                        info['structureUntilW4'] = structreUntilW4
                        info['structureUntilW5'] = structreUntilW5
                        info['sortedLengthStructureUntilW4'] = sortedLengthStructureUntilW4
                        info['sortedLengthStructureUntilW5'] = sortedLengthStructureUntilW5
                        info['highThreshold'] = highThreshold
                        info['downThreshold'] = downThreshold
                        info['numberOfAllStructureWithLengthBasedSortUntillW4'] = [int(item) for item in format(numberOfAllStructureWithLengthBasedSortUntillW4, '012b')]
                        info['numberOfAllStructureWithLengthBasedSortUntillW5'] = [int(item) for item in format(numberOfAllStructureWithLengthBasedSortUntillW5, '012b')]
                        self. dictForTrainInfo[f'{len(self. dictForTrainInfo)}'] = info

                        #######################################################
                    else:
                        W5Prime = self. dataCopy[self. selectedColumn].values[index]
                        self. pivotsIndexForPlot[-1] = index




    